In [199]:
import numpy as np
import pandas as pd
import os
import json

In [200]:
# These are the titles being parsed
titles = ['architect',
          'auditor',
          'business strategist',
          'recruiter',
          'software architect',
          'vp of engineering',
          'vp of marketing',
          'vp of operations',
          'analyst',
          'strategist',
          'associate'
          'manager',
          'director',
          'engineer',
          'associate',
          'scientist']

In [201]:
def job_extract(folder_profile_df, titles, destination_folder):
        """
        :param folder_profile_df: This folder consist of all the files parse
                                  from Indeed for different profiles
        :param titles: list of all the titles to be extracted
        :param destination_folder: This folder consist is the destination where
                                  the extracted data will be saved in the form of multiple files
        :return: saves the extracted data in the destination folder
        """
        with open(destination_folder + "data_extraction.json", 'w') as outfile:
            for file_item in os.listdir(folder_profile_df):
                raw_text = pd.read_csv(folder_profile_df + str(file_item))
                data = {}
                data["Job_Title"] = file_item.replace("_resume.csv", "")
                for i in range(raw_text.shape[0]):
                    work_exp_example = raw_text["work_exp"][i].replace("'",'"').\
                                       replace('u"','').replace('"]','').replace('[','').strip()
                    v = work_exp_example.lower().replace("\\u2022",'').replace("\\n",',').strip().split("\n")
                    v = [item.split(",") for item in v]
                    v = [item for sublist in v for item in sublist]
                    data["userID"] = (file_item + "_" + str(i)).replace("_resume.csv", "")
                    data["Experience"] = {}
                    ct, k = 0, 0
                    for item in v:
                        ct = ct + 1
                        if any(title in item for title in titles) and len(item) < 50:
                            k = k + 1
                            data["Experience"]["job" + str(k)] = {}
                            data["Experience"]["job" + str(k)]["designation"] = item.replace('"','').\
                                                                                          replace(']','').strip()
                            try:
                                if any(char.isdigit() for char in v[ct + 1]):
                                    data["Experience"]["job" + str(k)]["year"] = v[ct + 1].replace('"','').\
                                                                                          replace(']','').strip()
                                elif any(char.isdigit() for char in v[ct + 2]):
                                    data["Experience"]["job" + str(k)]["year"] = v[ct + 2].replace('"','').\
                                                                                          replace(']','').strip()
                                else:
                                    data["Experience"]["job" + str(k)]["year"] = "Year_Not_Found"
                            except:
                                data["Experience"]["job" + str(k)]["year"] = "Error"
                    data["Education"] = {}
                    degrees = raw_text["education_json"][i].replace("'",'"').replace('u"','').split(",")

                    years = raw_text["univ"][i].replace("'",'"').replace('u"','').replace('"','').split("\\n")[1:]

                    if len(degrees) == 0: 
                        data["Education"] = "degree not found"
                    elif len(years) == 0:
                        for i in range(len(degrees)):
                            data["Education"]["degree"+str(i+1)] = {}
                            data["Education"]["degree"+str(i+1)]["course"] = degrees[i].replace("degree_%s : "%str(i+1),"").\
                                                                            replace('"','').strip(),
                            data["Education"]["degree"+str(i+1)]["year"] = "Year not found"
                    else:
                        for i in range(len(degrees)):
                            data["Education"]["degree"+str(i+1)] = {}
                            data["Education"]["degree"+str(i+1)]["course"] = degrees[i].replace("degree_%s : "%str(i+1),"").\
                                                                        replace('"','').strip()
                            try:
                                data["Education"]["degree"+str(i+1)]["year"] = years[i].split(",")[0]
                            except:
                                data["Education"]["degree"+str(i+1)]["year"] = "Year not found"
                    outfile.write("%s\n" % data)

In [202]:
# home_path = "/Users/alvira/Desktop/spring2/app_develop/findYourPath"
folder_profile_df = "../Data/Profile_DFs/"
destination_folder = "../Data/Resume_cleaned/"
job_extract(folder_profile_df,titles, destination_folder)